# Tracking ML Experiment using MLFlow

We all have trained 1 model multiple times / say even whie tuning model - we may need to train it more than 10 times !!. If we try 10 diffeernt models. What if we can shuffle our data set we can create more differnt data sents and the number of experiments blow up. When we are wking in a team of data scientist if we want to track what happened in th nth experiment - ( what was the data / distibution/config/metrics ). Each Model run we are calling it an experimnet. Each experiment we need to capture Hyper parametrs / models / accuracy / charts/ AUC curve. This dahboard can help us track our expriments. This Dashboard help us track different experiments


Excel is a suboptimal approach. This can wer done with MLflow

Legal compliance - GDPR compliance 
1. organize all information neceaasry for experiment
2. Reproduce past runs and results using saved experiments
3. Log - Iterative improvements

In [17]:
import pandas as pd
import numpy as np

In [18]:
train_df=pd.read_csv("/workspaces/Experiment_Tracking_MLFlow/data/data.csv")
train_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP002529,Male,Yes,2,Graduate,No,6700,1750.0,230.0,300.0,1.0,Semiurban,Y
1,LP001385,Male,No,0,Graduate,No,5316,0.0,136.0,360.0,1.0,Urban,Y
2,LP001926,Male,Yes,0,Graduate,No,3704,2000.0,120.0,360.0,1.0,Rural,Y
3,LP001144,Male,Yes,0,Graduate,No,5821,0.0,144.0,360.0,1.0,Urban,Y
4,LP002562,Male,Yes,1,Not Graduate,No,5333,1131.0,186.0,360.0,NaN,Urban,Y


In [19]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            430 non-null    object 
 1   Gender             420 non-null    object 
 2   Married            427 non-null    object 
 3   Dependents         416 non-null    object 
 4   Education          430 non-null    object 
 5   Self_Employed      410 non-null    object 
 6   ApplicantIncome    430 non-null    int64  
 7   CoapplicantIncome  430 non-null    float64
 8   LoanAmount         414 non-null    float64
 9   Loan_Amount_Term   422 non-null    float64
 10  Credit_History     394 non-null    float64
 11  Property_Area      430 non-null    object 
 12  Loan_Status        430 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 43.8+ KB


## Binary Encoding of Categorical Variables

In [20]:
train_df['Gender']= train_df['Gender'].map({'Male':0, 'Female':1})
train_df['Married']= train_df['Married'].map({'No':0, 'Yes':1})
train_df['Loan_Status']= train_df['Loan_Status'].map({'N':0, 'Y':1})

## Checking for Missing Values

In [21]:
train_df.isnull().sum()

Loan_ID               0
Gender               10
Married               3
Dependents           14
Education             0
Self_Employed        20
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           16
Loan_Amount_Term      8
Credit_History       36
Property_Area         0
Loan_Status           0
dtype: int64

In [22]:
## dropping all the missing values
train_df = train_df.dropna()
train_df.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

## Segregating the target variable from the features

In [23]:
X = train_df[['Gender', 'Married', 'ApplicantIncome', 'LoanAmount', 'Credit_History']]
y = train_df.Loan_Status
X.shape, y.shape

((335, 5), (335,))

## Splitting the data

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=5)

## Model Training

In [25]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth=4, random_state=5)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=4, random_state=5)

## Cross Validation

In [26]:
from sklearn.metrics import accuracy_score

pred_val = model.predict(X_val)
accuracy_score(y_val, pred_val)

0.7313432835820896

In [27]:
pred_train = model.predict(X_train)
accuracy_score(y_train, pred_train)

0.8134328358208955

### We have created a model successfully, but now we have new set of data how can proceed on working with it

* we can change the data and run the code again
    * but we will loose the output and results from the old data
* we can create new cells below these to create a new model with the new data
    * but then when we have a lot of experiments in one file it will be really difficult finding the one we want to look at
* We can create new files for each experiment
    * but for actually comparing the results and outputs you'll still have to open each file and look into it closely

these are not the best ways of keeping track of the work and experiments that you perform, we need to create something that easy to manage, clearly shows the results and metrics, logs the changes and hyperparameters for us

# MLFlow open source platform for Machine learning life cycle.
 - Manage life cycle project
  - Track & store data
  - Store data, parameters, metrics, Artifacts

- All displayed in a web UI

- MLFLow is API based tool - logging metrics, code version, integrated with UI
- Each MLFlow Tracking session would store eveything for one single model Run
- 1 Run serves as execution of code we have written
- All runs which we run - will be stored in dir where code session is executed
- If we are working in a team it can be stored in S3 bucket / Host / on cloud
 

In [28]:
import mlflow
import os

- The problem when we are solving with random forest model will be 1 single experiment . We will have multiple runs. 
- Experiment: {Run, Run... Run}
- Experiment basically let us help us search for and visualise the run as well and download the run atrifact and meta data for analysis.

## Set Experiment
1. Set the name of the experient when we will perform task
2. All the details will be stored in YAML file


In [30]:
mlflow.set_experiment("loan_status_mlflow_exp_1")

2023/02/08 09:04:46 INFO mlflow.tracking.fluent: Experiment with name 'loan_status_mlflow_exp_1' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///workspaces/Experiment_Tracking_MLFlow/mlruns/710268977139061337', creation_time=1675847086404, experiment_id='710268977139061337', last_update_time=1675847086404, lifecycle_stage='active', name='loan_status_mlflow_exp_1', tags={}>

## There are 3 buckets we use to keep track of Experiments
- Parameters -> mlflow.log_params()
- Metrics -> mlflow.log_metric()
- Artifacts -> mlflow.log_artifact() 

#### Artifacts are any other files we wish to store. It can be Matrix, Plots, pickle model files

In [ ]:
with mlflow.start_run():
    model_rf= RandomForestClassifier(max_depth=4,random_state=5)
    model_rf.fit(X_train,y_train)

    ##capturing predicted values
    pred_val = model_rf.predict(X_val)
    val_acc = accuracy_score(y_val, pred_val)

    pred_train = model_rf(X_train)
    train_acc = accuracy_score(y_train,pred_train)

    